In [1]:
import json
import os.path
import time
import logging
import datetime
import itertools

import requests
import geojson
import tweepy

from tweepy import Stream
from tweepy.streaming import StreamListener

import IPython.display



logger = logging.getLogger('notebook')
logger.setLevel(logging.INFO)


In [2]:
# secret keys are stored here
keys = json.load(open(os.path.expanduser("~/.twitter.json")))['notebook-example']

# you can generate keys here:
# https://apps.twitter.com/

In [3]:
# now we can load the access keys
auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_key'], keys['access_secret'])
api = tweepy.API(auth)

In [4]:
# this should display your name
user = api.me()
user.name

'SiggyF'

In [5]:
public_tweets = api.home_timeline(count=2)
for tweet in public_tweets:
    print(tweet.created_at, tweet.text)
    

2017-11-30 07:31:17 DONDERDAG LIVE: Vanaf 10.15 uur Kamerdebat begroting @ministerieJenV met beantwoording bewindslieden… https://t.co/zhVCxJWNAW
2017-11-30 02:01:20 RT @chrisvfritz: PSA: The just-released @stylelint ^8.3.0 now supports &lt;style&gt; tags in .vue files out of the box, even with lang detection…


In [6]:
# display a tweet inline
IPython.display.HTML(api.get_oembed(id=tweet.id, hide_media=False)['html'])

In [7]:
counter = itertools.count()
tweets = api.search('#CoastSnapNarra', include_entities=True, count=100)
for tweet in tweets:
    print(tweet.created_at, tweet.text)
    
    for entity in tweet.entities['media']:
        resp = requests.get(entity['media_url'])
        # download the image
        with open('img-%06d.jpg' % next(counter), 'wb') as f:
            f.write(resp.content)
            
    

2017-11-22 15:08:33 RT @mikihirano: 本日の夕暮れ。#シドニー #オーストラリア #SnapSydney #CoastSnapNarra https://t.co/6DlvCiGSj1
2017-11-22 14:52:50 RT @mikihirano: #Narrabeen #beach #CoastSnapNarra #SnapSydney https://t.co/3o8f91HB72
2017-11-22 14:23:58 RT @mikihirano: #Narrabeen #beach #CoastSnapNarra #SnapSydney https://t.co/3o8f91HB72
2017-11-22 13:08:47 RT @mikihirano: #Narrabeen #beach #CoastSnapNarra #SnapSydney https://t.co/3o8f91HB72
2017-11-22 12:52:45 RT @mikihirano: 本日の夕暮れ。#シドニー #オーストラリア #SnapSydney #CoastSnapNarra https://t.co/6DlvCiGSj1
2017-11-22 11:50:49 本日の夕暮れ。#シドニー #オーストラリア #SnapSydney #CoastSnapNarra https://t.co/6DlvCiGSj1
2017-11-22 10:44:53 RT @mikihirano: #Narrabeen #beach #CoastSnapNarra #SnapSydney https://t.co/3o8f91HB72
2017-11-22 07:29:43 RT @mikihirano: #Narrabeen #beach #CoastSnapNarra #SnapSydney https://t.co/3o8f91HB72
2017-11-22 07:01:10 #Narrabeen #beach #CoastSnapNarra #SnapSydney https://t.co/3o8f91HB72


In [12]:
# now let's start listening for events
class PrintListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status.text)
        
print_listener = PrintListener()
stream = Stream(api.auth, print_listener)
stream.filter(track=['flooded'], async=True)

Pull out or get flooded 🤔 Ima flood ya baby girl 🍆💦💦🌊
Aerial View Shows Colombian Town Flooded With Mud. Credit – Fuerza Aérea Colombiana via Storyful - https://t.co/gpRplXM7PW via @AUNewsToday


In [13]:
stream.disconnect()

In [ ]:
api.